# Hyperparameter Optimization

To optimize the hyperparameters of the inference and summary networks, we use [Optuna](https://www.optuna.org), a framework-agnostic hyperparameter optimization library that efficiently minimizes custom objective functions to identify optimal parameter settings. Our optimization process involves four three steps:

1. **Data Simulation.** We simulate a large *training data set* that will be used throughout all subsequent training phases as well as a smaller *validation data set* that will be used to assess the outcome metric.

2. **Defining the Objective Function and Search Space.** The objective function takes a set of hyperparameters, evaluates the outcome of interest and returns a single value. In our case, the function includes a whole offline training phase on the *training data set* and computes a weighted sum of metrics capturing parameter recovery, simulation-based calibration, and posterior contraction with respect to the *validation data set*. Within the objective function, the search space has to be defined by specifying a range of possible values for each hyperparameter.


3. **Run Optuna Trials**. During multiple trials, Optuna iteratively proposes and evaluates hyperparameter sets to minimize the objective function. Each trial involves one execution of the objective function. In our case, that is a whole offline training procedure on the same pre-simulated *training data set* and the computation of the weighted sum applying the trained network on the *validation data set*. Based on this score, Optuna updates the parameter proposals for the next trial, which ideally leads to a stepwise minimization of the objective function.

After running all trials, we can identify which parameter configurations yielded the lowest objective value and visualize the optimization results. 

Before starting with these steps, we have to load all libraries:

In [ ]:
import sys
sys.path.append("../../BayesFlow")
sys.path.append("../")

import os
import torch 

# optional if you have a GPU:
#print("CUDA available:", torch.cuda.is_available(), flush=True)
#print(torch.cuda.device_count(), flush=True)
#print("Using device:", torch.cuda.get_device_name(0))

if "KERAS_BACKEND" not in os.environ:
    # set this to "torch", "tensorflow", or "jax"
    os.environ["KERAS_BACKEND"] = "torch"

import numpy as np
import pickle

import keras

import optuna

import bayesflow as bf



/home/administrator/miniforge3/envs/bf_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:bayesflow:Using backend 'torch'
When using torch backend, we need to disable autograd by default to avoid excessive memory usage. Use

with torch.enable_grad():
    ...

in contexts where you need gradients (e.g. custom training loops).


## Data Simulation

To train and evaluate our models, we simulate reaction time (RT) and accuracy data using the Diffusion Model for Conflict Tasks (DMC), as described in the [DMC Introduction](./dmc_introduction.ipynb).

To ensure consistent dataset sizes, we set the `fixed_num_obs` parameter. Without this, the simulator would generate datasets with a random number of trials that stays consistent across all data sets. This could bias the optimization process toward hyperparameter settings that are overly specific to a particular trial count. Therefore, it’s important to select a `fixed_num_obs` value that closely reflects the characteristics of the target application. We aimed to apply the hyperparameter setting to a varying trial number between 50 and 1000 and therefore used a medium number of 500 trials for the hyperparameter optimization.

In [48]:
from dmc import DMC, dmc_helpers

simulator = DMC(prior_means=np.array([70.8, 114.71, 0.71, 332.34, 98.36, 43.36]),
                prior_sds=np.array([19.42, 40.08, 0.14, 52.07, 30.05, 9.19]),
                sdr_fixed=None,
                tmax=1500, 
                contamination_probability=None,
                fixed_num_obs=500,
                param_names=("A", "tau", "mu_c", "mu_r", "b", "sd_r"))

We are now able to simulate multiple data sets efficiently by simply running:

In [49]:
n_data_sets = 10

test_data = simulator.sample(n_data_sets)

We can inspect the shapes of the dataset to verify that the simulation ran correctly:

In [50]:
for key, i in test_data.items():
    print(f'{key}: {i.shape}')

A: (10, 1)
tau: (10, 1)
mu_c: (10, 1)
mu_r: (10, 1)
sd_r: (10, 1)
b: (10, 1)
rt: (10, 500, 1)
accuracy: (10, 500, 1)
conditions: (10, 500, 1)
num_obs: (10, 1)


For the optimization algorithm, we simulated 50,000 data sets for the training data set and 1000 data sets for the validation data set. However, for demonstration purposes we only simulate 100 training data sets and 100 validation data sets:

In [24]:
train_data = simulator.sample(100)

val_data = simulator.sample(100)

## Defining the Objective Function and Search Space

The **objective function** is the core component of the Optuna optimization process. It defines the discrepancy measure that the algorithm seeks to minimize across multiple training iterations. Following the approach described in Schaefer et al. (2025), we employ a weighted sum of three key metrics: 

1. **Normalized Root Mean Squared Error (NRMSE)** – Measures parameter recovery accuracy, i.e., how well the inferred parameters match the true parameters across simulations. Lower NRMSE indicates better recovery.

2. **Expected Calibration Error (ECE)** – Evaluates simulation-based calibration. A smaller ECE means the approximated posterior distributions are better calibrated.

2. **Contraction Factor (CF)** – Reflects the Posterior Contraction (PC), i.e., how concentrated the posterior is around the true parameters compared to the prior variance. A higher Posterior Contraction indicates higher reduction of uncertainty. To allign the direction of interpretation with that of the remaining metrics, we compute CF as the inverse Posterior Contraction: $1-PC$. That is, a low CF indicates a high reduction of uncertainty.

We compute the weighted sum as:

\begin{equation}
L = w_{1} \overline{\text{NRMSE}} + w_2 \overline{\text{\text{ECE}}} + w_3 \overline{\text{CF}} 
\end{equation}

The objective function executes the following steps:

1. **Define Search Space**. We have to suggest values for each specified parameter. We can use the functions 
  * `trial.suggest_float()` for continous variables, 
  * `trial.suggest_int()` for integer variables (chooses values between a minimum and maximum) and 
  * `trial.suggest_categorical` (suggest only predefined levels). 
2. **Define Inference and Summary Networks**. Each network is set up using the suggested values as hyperparameters. 
3. **Train Networks**. The networks are trained in an offline training procedure using the suggest values.
4. **Compute weighted sum L**. We first compute the default diagnostics (NRMSE, ECE and PC). Based on these metrics, we compute the weighted sum L using the function `dmc_helpers.weighted_metric_sum()`. Note that the objective function only returns a single value for L.

In earlier training phases, we observed that the loss for our model reaches a relatively low level after about 50 epochs, with only marginal improvements beyond that point. Taken into account that the training phase will be repeated in each Optuna-trial, we decided to choose this number. However, for demonstration, we only run 10 training epochs:

In [ ]:
def objective(trial, epochs=10):

    # Define Search Space
    dropout = trial.suggest_float("dropout", 0.01, 0.3)
    initial_learning_rate = trial.suggest_float("lr", 1e-4, 1e-3) 
    num_seeds = trial.suggest_int("num_seeds", 1, 8)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
    embed_dim = trial.suggest_categorical("embed_dim", [64, 128])
    summary_dim = trial.suggest_int("summary_dim", 4, 32)

    # Define the adapter
    adapter = (
        bf.adapters.Adapter()
        .convert_dtype("float64", "float32")
        .sqrt("num_obs")
        .concatenate(["A", "tau", "mu_c", "mu_r", "b", 'sd_r'], into="inference_variables")
        .concatenate(["rt", "accuracy", "conditions"], into="summary_variables")
        .standardize(include="inference_variables")
        .rename("num_obs", "inference_conditions")
        )

    # Define inference net 
    inference_net = bf.networks.FlowMatching(coupling_kwargs=dict(subnet_kwargs=dict(dropout=dropout)))

    # Define summary net
    summary_net = bf.networks.SetTransformer(summary_dim=summary_dim, num_seeds=num_seeds, dropout=dropout, embed_dim=(embed_dim, embed_dim))
    
    workflow = bf.BasicWorkflow(
        simulator=simulator,
        adapter=adapter,
        initial_learning_rate=initial_learning_rate,
        inference_network=inference_net,
        summary_network=summary_net,
        checkpoint_filepath= '../data_complete/optuna_checkpoints',
        checkpoint_name= f'network_{round(dropout, 2)}_{round(initial_learning_rate, 2)}_{num_seeds}_{batch_size}_{embed_dim}_{summary_dim}',
        inference_variables=["A", "tau", "mu_c", "mu_r", "b", 'sd_r'])
    
    # Start Offline Training Procedure
    history = workflow.fit_offline(train_data, epochs=epochs, batch_size=batch_size, validation_data=val_data, verbose=0)
    
    # Compute Metrics Recovery, SBC, PC
    metrics_table=workflow.compute_default_diagnostics(test_data=val_data)

    # Computed weighted sum L
    L = dmc_helpers.weighted_metric_sum(metrics_table, weight_recovery=1, weight_pc=1, weight_sbc=1)
    
    return L


We now create a study object:

In [30]:

study = optuna.create_study(direction="minimize")


[I 2025-07-18 10:30:04,645] A new study created in memory with name: no-name-71fe456a-2ee9-4b1f-be3c-32238f2048c2


We can now execute the study across multiple trials. Be aware that this process can be computationally intensive, and runtime will vary depending on the hardware setup. In Schaefer et al. (2025), the optimization involved 50 trials using 50 training epochs over 50,000 training datasets and 1,000 validation datasets, with the full computation taking approximately 45 hours on an NVIDIA A100 SXM4 80 GB GPU. For demonstration purposes here, we limit the run to 10 trials to keep the runtime manageable:

In [31]:

study.optimize(objective, n_trials=10)

trial = study.best_trial
print("Outcome Metric: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))



INFO:bayesflow:Fitting on dataset instance of OfflineDataset.
INFO:bayesflow:Building on a test batch.
INFO:bayesflow:Training is now finished.
            You can find the trained approximator at '../data_complete/optuna_checkpoints/network_0.17_0.0_8_32_64_13.network_0.17_0.0_8_32_64_13.keras'.
            To load it, use approximator = keras.saving.load_model(...).
[I 2025-07-18 10:31:46,174] Trial 0 finished with value: 1.5426481802641618 and parameters: {'dropout': 0.17131354078376448, 'lr': 0.00045175605420312196, 'num_seeds': 8, 'batch_size': 32, 'embed_dim': 64, 'summary_dim': 13}. Best is trial 0 with value: 1.5426481802641618.
INFO:bayesflow:Fitting on dataset instance of OfflineDataset.
INFO:bayesflow:Building on a test batch.
INFO:bayesflow:Training is now finished.
            You can find the trained approximator at '../data_complete/optuna_checkpoints/network_0.24_0.0_8_32_64_12.network_0.24_0.0_8_32_64_12.keras'.
            To load it, use approximator = keras.saving.l

Outcome Metric: 1.4344895329407834
Best hyperparameters: {'dropout': 0.26865304748694374, 'lr': 0.0005948312875696228, 'num_seeds': 4, 'batch_size': 16, 'embed_dim': 128, 'summary_dim': 23}


After running the study, we can inspect the search space by plotting the objective function as a function of parameters:

In [51]:
from plotly.io import show

fig = optuna.visualization.plot_slice(study, params=["dropout", "lr", 'num_seeds', 'batch_size', 'embed_dim', 'summary_dim'])
show(fig)